### Paper:
   #### Classification of Malicious Web Code Using Deep Learning [Manoj Kumar Selvam]
### Dataset:
    Total data 10858
    XSS : 7121 + 231 = 7352
    Benign : 3066 + 440 = 3506
   
   

In [138]:
import numpy as np
import pandas as pd
import glob
import time
import pandas as pd
# from xml.dom import minidom

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords 


import os
import matplotlib.pyplot as plt
import keras

[nltk_data] Downloading package punkt to /home/ebryx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ebryx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [139]:
import cv2

In [140]:
path='./data/portswigger.txt'

XSS=[]
f = open(path, "r")
for x in f:
    XSS.append(x)

In [141]:
# Manually removed tag data


path='./data/'
file="scottparker_ml_sqli_src__trainingdata_plain.txt"

In [142]:
df = pd.read_csv(os.path.join(path,file), sep='Aw3s0meSc0t7', names=['benign'], header=None, engine='python')

In [143]:
df.head()

,benign
0,Add plain text here
1,Ne te quaesiveris extra.
2,Man is his own star; and the soul that can
3,"Render an honest and a perfect man,"
4,"Commands all light, all influence, all fate;"


In [144]:
plain_text=df['benign'].values

In [145]:
plain_text[:5]

array(['Add plain text here', 'Ne te quaesiveris extra.',
       'Man is his own star; and the soul that can',
       'Render an honest and a perfect man,',
       'Commands all light, all influence, all fate;'], dtype=object)

In [146]:
plain_text=plain_text[:-22] # removed last 22 records that were urls

In [147]:
len(plain_text)

3641

In [148]:
data=''
for x in plain_text:
    data+=" " + x

In [149]:
stop_words = set(stopwords.words('english')) 
def fun_remove_stop_words(posts):
    filtered=''
    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x
    return filtered

In [150]:
data=fun_remove_stop_words(data)
data=data.split('.')

In [151]:
data[:5]

['  Add plain text Ne te quaesiveris extra',
 ' Man star; soul Render honest perfect man, Commands light, influence, fate; Nothing falls early late',
 ' Our acts angels are, good ill, Our fatal shadows walk us still',
 ' Epilogue Beaumont Fletcher’s Honest Man’s Fortune Cast bantling rocks, Suckle she-wolf’s teat; Wintered hawk fox, Power speed hands feet',
 ' I read day verses written eminent painter original conventional']

In [152]:
len(data)

3066

In [153]:
## Additional benign data 

path='./data/benign_for_training.txt'
benign_data=[]
f = open(path, "r")
for x in f:
    benign_data.append(x)

In [161]:
len(benign_data)

441

In [154]:
benign_sentence=[]
for i in benign_data:
    sentences=i.split('.')
    
    for sentence in sentences:
        benign_sentence.append(sentence)

In [155]:
len(benign_sentence)

746

In [156]:
path='./data/xss_for_training.txt'
xss_data=[]
f = open(path, "r")
for x in f:
    xss_data.append(x)

In [157]:
values=[]
for i in XSS:
    values.append((i,1))

In [158]:


for i in data:
    values.append((i,0))



In [159]:
for i in benign_sentence:
    values.append((i,0))

In [160]:
for i in xss_data:
    values.append((i,1))

In [162]:
len(xss_data)

280

In [163]:
len(XSS)

7121

In [164]:
df=pd.DataFrame(values,columns=['Sentence','Label'])

In [165]:
df.to_csv('XSS_dataset.csv', encoding='utf-8-sig')

In [166]:
len(values)

11213

In [167]:
values[1]

('<a draggable="true" ondragend="alert(1)">test</a>\n', 1)

In [168]:
values[-1]

("<script> alert(asdal'') </script>\n", 1)

In [169]:
sentences=df['Sentence'].values
sentences[1]

'<a draggable="true" ondragend="alert(1)">test</a>\n'

In [170]:
def convert_to_ascii(sentence):
    sentence_ascii=[]

    for i in sentence:
        sentence_ascii.append(ord(i))

    zer=np.zeros((10000))

    for i in range(len(sentence_ascii)):
        zer[i]=sentence_ascii[i]

    zer.shape=(100, 100)


#     plt.plot(image)
#     plt.show()
    return zer


In [172]:
# display_ascii(sentences[100])


arr=np.zeros((11213,100,100))

for i in range(len(sentences)):
    
    image=convert_to_ascii(sentences[i])

    x=np.asarray(image,dtype='float')
    image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    image/=128
    
#     if i==1:
#         plt.plot(image)
#         plt.show()    
    arr[i]=image

In [173]:
arr.shape

(11213, 100, 100)

In [174]:
data = arr.reshape(arr.shape[0], 100, 100, 1)

In [175]:
data.shape

(11213, 100, 100, 1)

In [176]:
y=df['Label'].values

In [177]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(data,y, test_size=0.2, random_state=42)

In [178]:


from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D, BatchNormalization



In [179]:

model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu, input_shape=(100,100,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])

In [180]:


model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 98, 98, 64)        640       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 47, 47, 128)       73856     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 23, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 67712)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               17334528  
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

In [181]:
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.96):
            print(" \n Reached 96% + validation accuracy")
            self.model.stop_training=True
callbacks = myCallback()

In [182]:
batch_size = 128
num_epoch = 10
#model training
model_log = model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=( testX,  testY),
                      callbacks=[callbacks])

Train on 8970 samples, validate on 2243 samples
Epoch 1/10
8970/8970 [==============================] - 78s 9ms/sample - loss: 0.5365 - accuracy: 0.7361 - val_loss: 0.4034 - val_accuracy: 0.7954
Epoch 2/10
8960/8970 [============================>.] - ETA: 0s - loss: 0.2010 - accuracy: 0.9281 
 Reached 96% + validation accuracy
8970/8970 [==============================] - 78s 9ms/sample - loss: 0.2008 - accuracy: 0.9282 - val_loss: 0.0955 - val_accuracy: 0.9746


In [183]:
pred=model.predict(testX)

In [184]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [185]:
true=0
false=0

for i in range(len(pred)):
    if pred[i] == testY[i]:
        true+=1
    else:
        false+=1

In [186]:
print("correct predicted :: ", true)

correct predicted ::  2186


In [187]:
print("false prediction :: ", false)

false prediction ::  57


In [188]:
attack=0
benign=0
for i in range(len(testY)):
    if testY[i]==1:
        attack+=1
    else:
        benign+=1

In [189]:
print("Attack data in test set :: ", attack)

Attack data in test set ::  1507


In [190]:
print(" Benign data in test set :: ", benign)

 Benign data in test set ::  736


## Confusion Matrix

In [191]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy


In [192]:
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision


In [193]:


def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall



In [194]:
def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [195]:
accuracy,precision,recall=confusion_matrix(testY,pred)

In [196]:
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))


 Accuracy : 0.9745876058849755 
 Precision : 0.9945429740791268 
 Recall : 0.9674850696748507


In [197]:
from sklearn.metrics import precision_score
precision_score(testY, pred)

0.9945429740791268

In [198]:
from sklearn.metrics import recall_score
recall_score(testY, pred)


0.9674850696748507

### Save Model

In [199]:
from keras.models import load_model

model.save('my_model_cnn.h5')


#### Testing

In [200]:
from keras.models import load_model




mymodel = tf.keras.models.load_model('my_model_cnn.h5')

In [201]:
import numpy as np
import cv2
def predict_cross_site_script():
    
    repeat=True
    
    
    
    
    beautify=''
    for i in range(20):
        beautify+= "="

    print(beautify) 
    input_val=input("Give me some data to work on : ")
    print(beautify)
    
    if input_val== '0':
        repeat=False
        
        
    
    if repeat == True:

        sentence_ascii=[]

        for i in input_val:
            sentence_ascii.append(ord(i))

        storage=np.zeros((10000))

        for i in range(len(sentence_ascii)):
            storage[i]=sentence_ascii[i]

        storage.shape=(100, 100)

        x=np.asarray(storage,dtype='float')
        image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
        image.shape=(1,100,100,1)
        image/=128

        prediction=mymodel.predict(image)

        if prediction>0.5:

    #         print(f"Chances of attack :  {prediction[0]*100} ")
            print(" It can be Cross site scripting attack")

        else:

    #         print(f" Chances of being safe {100 - (prediction*100) }")
            print("seems safe")
        
        
        predict_cross_site_script()
        
        
    elif repeat == False:
        print( " Good Bye ")

In [7]:
# Test for XSS cheatsheet

predict_cross_site_script()

Give me some data to work on : <a id=x tabindex=1 onactivate=alert(1)></a>
 It can be Cross site scripting attack
Give me some data to work on : <body onafterprint=alert(1)>
 It can be Cross site scripting attack
Give me some data to work on : <style>@keyframes x{from {left:0;}to {left: 1000px;}}:target {animation:10s ease-in-out 									0s 1 x;}</style><a id=x style="position:absolute;" onanimationcancel="alert(1)"></a>
 It can be Cross site scripting attack
Give me some data to work on : <style>@keyframes x{}</style><a style="animation-name:x" 									onanimationend="alert(1)"></a>
 It can be Cross site scripting attack
Give me some data to work on : <style>@keyframes slidein {}</style><a style="animation-duration:1s;animation-name:slidein;animation-iteration-count:2" 									onanimationiteration="alert(1)"></a>
 It can be Cross site scripting attack
Give me some data to work on : <style>@keyframes x{}</style><a style="animation-name:x" 									onanimationstart="alert(1)"></a>
 I

 It can be Cross site scripting attack
Give me some data to work on : <article onbeforepaste="alert(1)" contenteditable>test</article>
 It can be Cross site scripting attack
Give me some data to work on : <article onpaste="alert(1)" contenteditable>test</article>
 It can be Cross site scripting attack
Give me some data to work on : <aside draggable="true" ondragleave="alert(1)">test</aside>
 It can be Cross site scripting attack
Give me some data to work on : <aside id=x tabindex=1 onbeforeactivate=alert(1)></aside>
 It can be Cross site scripting attack
Give me some data to work on : <aside onbeforepaste="alert(1)" contenteditable>test</aside>
 It can be Cross site scripting attack
Give me some data to work on : onclick="alert(1)"
 It can be Cross site scripting attack
Give me some data to work on : alert(1)
seems safe
Give me some data to work on : <audio autoplay onplay=alert(1)><source src="validaudio.wav" type="audio/wav"></audio>
 It can be Cross site scripting attack
Give me som

Give me some data to work on : <rt onmousedown="alert(1)">test</rt>
 It can be Cross site scripting attack
Give me some data to work on : <s onblur=alert(1) tabindex=1 id=x></s><input autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <script draggable="true" ondragstart="alert(1)">test</script>
 It can be Cross site scripting attack
Give me some data to work on : <script oncut="alert(1)" contenteditable>test</script>
 It can be Cross site scripting attack
Give me some data to work on : <section onmouseup="alert(1)">test</section>
 It can be Cross site scripting attack
Give me some data to work on : <shadow id=x tabindex=1 onfocusin=alert(1)></shadow>
 It can be Cross site scripting attack
Give me some data to work on : <slot draggable="true" ondragstart="alert(1)">test</slot>
 It can be Cross site scripting attack
Give me some data to work on : <small onkeyup="alert(1)" contenteditable>test</small>
 It can be Cross site scripting attack
Give me some data 

In [136]:
predict_cross_site_script()

Give me some data to work on : is this attack
seems safe
Give me some data to work on : am i saqlain hussain shah
seems safe
Give me some data to work on : document.getElementsByClassName("example")
 It can be Cross site scripting attack
Give me some data to work on : document.write()
seems safe
Give me some data to work on : document.getElementsByName("")
 It can be Cross site scripting attack
Give me some data to work on : document.querySelector(".example")
 It can be Cross site scripting attack
Give me some data to work on : document.write("<h1>Hello World</h1>")
 It can be Cross site scripting attack
Give me some data to work on : document.close()
seems safe
Give me some data to work on : <style>:target {transform: rotate(180deg);}</style><a id=x style="transition:transform
 It can be Cross site scripting attack
Give me some data to work on : <body onhashchange="alert(1)">
seems safe
Give me some data to work on : onhashchange="alert(1)"
seems safe
Give me some data to work on : <s

In [137]:
predict_cross_site_script()

Give me some data to work on : ontransitionrun=alert(1)
 It can be Cross site scripting attack
Give me some data to work on : <video onmousemove="alert(1)">test</video>
 It can be Cross site scripting attack
Give me some data to work on : <wbr onclick="alert(1)">test</wbr>
 It can be Cross site scripting attack
Give me some data to work on : <wbr onpointerleave=alert(1)>XSS</wbr>
 It can be Cross site scripting attack
Give me some data to work on : <xss id=x tabindex=1 ondeactivate=alert(1)></xss><input autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <var onblur=alert(1) tabindex=1 id=x></var><input autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <u onpointermove=alert(1)>XSS</u>
 It can be Cross site scripting attack
Give me some data to work on : <u draggable="true" ondragstart="alert(1)">test</u>
 It can be Cross site scripting attack
Give me some data to work on : <tt oncontextmenu="alert(1)">test</tt>
 It can be Cro

 It can be Cross site scripting attack
Give me some data to work on : <table onkeydown="alert(1)" contenteditable>test</table>
 It can be Cross site scripting attack
Give me some data to work on : <table draggable="true" ondragenter="alert(1)">test</table>
 It can be Cross site scripting attack
Give me some data to work on : <svg><video onload=alert(1)></video>
 It can be Cross site scripting attack
Give me some data to work on : <svg><thead onload=alert(1)></thead>
 It can be Cross site scripting attack
Give me some data to work on : <svg><textarea onload=alert(1)></textarea>
 It can be Cross site scripting attack
Give me some data to work on : <svg><table onload=alert(1)></table>
 It can be Cross site scripting attack
Give me some data to work on : <svg><slot onload=alert(1)></slot>
 It can be Cross site scripting attack
Give me some data to work on : <svg><path><animateMotion onbegin=alert(1) dur="1s" repeatCount="1">
 It can be Cross site scripting attack
Give me some data to work 

Give me some data to work on : <ins onblur=alert(1) tabindex=1 id=x></ins><input autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <input4 onpointerout=alert(1)>XSS</input4>
 It can be Cross site scripting attack
Give me some data to work on : <input4 onpointerup=alert(1)>XSS</input4>
 It can be Cross site scripting attack
Give me some data to work on : <input3 onpointerenter=alert(1)>XSS</input3>
 It can be Cross site scripting attack
Give me some data to work on : <input type=image src=validimage.png onload=alert(1)>
 It can be Cross site scripting attack
Give me some data to work on : <input onpointerleave=alert(1)>XSS</input>
 It can be Cross site scripting attack
Give me some data to work on : <input onfocusout=alert(1) id=x><input autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <input onbeforepaste=alert(1) value="" autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <input id=x tabindex=